# Feature Engineering

In [195]:
import pandas as pd 
import numpy as np 
import os
import sys

# Get the absolute path to the 'util' directory (assuming 'eda.ipynb' is one level down from 'your_project')
data_dir = os.path.abspath(os.path.join('Documents/CaliforniaWildfireAnalysis', 'data'))

# Add the 'util' directory to sys.path if it's not already there
if data_dir not in sys.path:
    sys.path.append(data_dir)
data_dir

'/Users/shireen/Documents/CaliforniaWildfireAnalysis/data'

In [198]:
df = pd.read_csv(data_dir+'/supervised_leaned_eda.csv')
df.head()

,ZIP Code,Cov A Amount Weighted Avg,Avg Fire Risk Score,Number of Negligible Fire Risk Exposure,Number of Low Fire Risk Exposure,Number of Moderate Fire Risk Exposure,Number of High Fire Risk Exposure,Number of Very High Fire Risk Exposure,Earned Premium 2020,Earned Exposure 2020,Non-CAT Cov A Fire - Incurred Losses,Non-CAT Cov A Smoke - Incurred Losses,Non-CAT Cov A Smoke - Number of Claims,Non-CAT Cov C Fire - Number of Claims,Non-CAT Cov C Smoke - Incurred Losses,CAT Cov A Smoke - Number of Claims,CAT Cov C Fire - Incurred Losses,CAT Cov C Smoke - Incurred Losses,Earned Premium 2021,Earned Exposure 2021
0,90001,261658.94,0.32,884,407,0,0,0,982193,1291,96882,9320,1,1,5070,13,0,0,1076066,1345
1,90002,250804.27,0.33,1270,614,0,0,0,1400005,1884,45212,1686,1,1,542,5,3582,0,1523488,1939
2,90003,266381.48,0.31,1195,535,1,0,0,1424103,1731,449601,0,0,2,0,6,6800,223,1537173,1769
3,90004,929430.46,0.45,962,643,56,0,0,3992219,1661,7635,0,0,0,0,1,0,384,4428387,1675
4,90005,1329088.35,0.44,224,127,16,0,0,1263229,368,1383,0,0,0,0,0,0,0,1377640,379


In [210]:
df["Total CAT Losses"] = df["CAT Cov C Fire - Incurred Losses"]+ df["CAT Cov C Smoke - Incurred Losses"]
df["Total Non-CAT Losses"] = df["Non-CAT Cov A Fire - Incurred Losses"] + df["Non-CAT Cov A Smoke - Incurred Losses"]
df.rename(columns = { "CAT Cov A Smoke - Number of Claims":"Total CAT Claims"}, inplace= True)
df["Total Non-CAT Claims"] = df["Non-CAT Cov A Smoke - Number of Claims"]+df["Non-CAT Cov C Fire - Number of Claims"]

In [212]:
df.columns

Index(['ZIP Code', 'Cov A Amount Weighted Avg', 'Avg Fire Risk Score',
       'Number of Negligible Fire Risk Exposure',
       'Number of Low Fire Risk Exposure',
       'Number of Moderate Fire Risk Exposure',
       'Number of High Fire Risk Exposure',
       'Number of Very High Fire Risk Exposure', 'Earned Premium 2020',
       'Earned Exposure 2020', 'Non-CAT Cov A Fire - Incurred Losses',
       'Non-CAT Cov A Smoke - Incurred Losses',
       'Non-CAT Cov A Smoke - Number of Claims',
       'Non-CAT Cov C Fire - Number of Claims',
       'Non-CAT Cov C Smoke - Incurred Losses', 'Total CAT Claims',
       'CAT Cov C Fire - Incurred Losses', 'CAT Cov C Smoke - Incurred Losses',
       'Earned Premium 2021', 'Earned Exposure 2021', 'Total CAT Losses',
       'Total Non-CAT Losses', 'Total Non-CAT Claims'],
      dtype='object')

In [222]:
df['Avg Premium'] = df['Earned Premium 2020'] / df['Earned Exposure 2020'].replace(0, np.nan)
df['Avg CAT Loss'] = df['Total CAT Losses'] / df['Earned Exposure 2020'].replace(0, np.nan)
df['Avg Non-CAT Loss'] = df['Total Non-CAT Losses'] / df['Earned Exposure 2020'].replace(0, np.nan)
df['Avg CAT Claims'] = df['Total Non-CAT Claims'] / df['Earned Exposure 2020'].replace(0, np.nan)
df['Avg Non-CAT Claims'] = df['Total CAT Claims'] / df['Earned Exposure 2020'].replace(0, np.nan)
df['Avg Premium 2021'] = df['Earned Premium 2021'] / df['Earned Exposure 2021'].replace(0, np.nan)
df['Loss Ratio'] = (df['Total CAT Losses'] + df['Total Non-CAT Losses'])/df['Earned Premium 2020'].replace(0, np.nan)
df['Claim Frequency'] = (df['Total Non-CAT Claims']+df['Total CAT Claims'])/df['Earned Exposure 2020'].replace(0, np.nan)
df['Average Claim Severity'] = (df['Total CAT Losses'] + df['Total Non-CAT Losses'])/(df['Total Non-CAT Claims']+df['Total CAT Claims']).replace(0, np.nan)
# Replace inf with NaN, then drop all NaN values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [223]:
df["Avg Fire Risk Score"]=df["Avg Fire Risk Score"].round(0)
df.head()

,ZIP Code,Cov A Amount Weighted Avg,Avg Fire Risk Score,Number of Negligible Fire Risk Exposure,Number of Low Fire Risk Exposure,Number of Moderate Fire Risk Exposure,Number of High Fire Risk Exposure,Number of Very High Fire Risk Exposure,Earned Premium 2020,Earned Exposure 2020,...,Total Non-CAT Claims,Avg Premium,Avg CAT Loss,Avg Non-CAT Loss,Avg CAT Claims,Avg Non-CAT Claims,Avg Premium 2021,Loss Ratio,Claim Frequency,Average Claim Severity
0,90001,261658.94,0.0,884,407,0,0,0,982193,1291,...,2,760.800155,0.000000,82.263362,0.001549,0.010070,800.049071,0.108127,0.011619,7080.133333
1,90002,250804.27,0.0,1270,614,0,0,0,1400005,1884,...,2,743.102442,1.901274,24.892781,0.001062,0.002654,785.708097,0.036057,0.003715,7211.428571
2,90003,266381.48,0.0,1195,535,1,0,0,1424103,1731,...,2,822.705373,4.057192,259.734835,0.001155,0.003466,868.950254,0.320640,0.004622,57078.000000
3,90004,929430.46,0.0,962,643,56,0,0,3992219,1661,...,0,2403.503311,0.231186,4.596629,0.000000,0.000602,2643.813134,0.002009,0.000602,8019.000000
5,90006,463506.41,0.0,220,121,1,0,0,428828,341,...,3,1257.560117,50.466276,1989.835777,0.008798,0.000000,1346.379888,1.622429,0.008798,231914.333333


## Feature Selection

In [233]:
features_to_drop = highly_correlated_features(df, "Avg Premium 2021")
features_to_drop

['Earned Premium 2021',
 'Earned Exposure 2021',
 'CAT Cov C Fire - Incurred Losses',
 'Non-CAT Cov A Fire - Incurred Losses',
 'Non-CAT Cov A Smoke - Number of Claims',
 'Avg Non-CAT Loss',
 'Avg Premium 2021']